In [1]:
import sys
import os
import json
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, FloatSlider, Button, HBox, VBox
from IPython.display import display, clear_output, Image
import matplotlib.pyplot as plt
from IPython.display import HTML, display, clear_output, Image
import tempfile
import time

# Adiciona a pasta 'src' ao caminho do Python
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from simulation import run_simulation, save_results, load_parameters
from utils import plot_sir_curves, save_grid_animation, calculate_box_counting_dimension
from model import EpidemiaCA

In [2]:
default_params_path = '../data/raw/parameters.json'
default_params = load_parameters(default_params_path)

def run_simulation_and_display(params):
    
    # A mensagem de "carregando" será exibida e, em seguida,
    # esta função irá executar o código abaixo.

    history, state_counts = run_simulation(params)
    
    with tempfile.TemporaryDirectory() as temp_dir:
        # Salva os gráficos e o GIF na pasta temporária
        plot_sir_curves(pd.DataFrame(state_counts), temp_dir)
        save_grid_animation(history, temp_dir)

        # Limpa a mensagem de "Simulação em andamento..."
        clear_output(wait=True)
        
        # Exibe os resultados finais
        print(f"Simulação concluída com os parâmetros: {params}")
        
        # Exibe a grade final
        plt.figure(figsize=(6, 6))
        plt.imshow(history[-1], cmap='viridis', vmin=0, vmax=2)
        plt.title(f"Estado Final da Simulação após {params['num_steps']} passos")
        plt.show()

        # Exibe as curvas SIR
        sir_image_path = os.path.join(temp_dir, 'sir_curves.png')
        display(Image(filename=sir_image_path))

        # Exibe a animação
        animation_path = os.path.join(temp_dir, 'simulation.gif')
        display(Image(filename=animation_path))
        
        # Calcula e imprime a dimensão fractal
        peak_time = pd.DataFrame(state_counts)['infected'].idxmax()
        grid_at_peak = history[peak_time]
        dim = calculate_box_counting_dimension(grid_at_peak)
        print(f"Dimensão de box-counting no pico da epidemia: {dim:.4f}")

In [3]:
# Célula [3] do notebook: Sliders, botão e lógica de exibição

# Sliders
grid_size_slider = IntSlider(min=50, max=200, step=10, value=default_params['grid_size'], description='Tamanho da Grade')
initial_infected_slider = IntSlider(min=1, max=100, step=1, value=default_params['initial_infected'], description='Infectados Iniciais')
infection_rate_slider = FloatSlider(min=0.01, max=1.0, step=0.01, value=default_params['infection_rate'], description='Taxa de Infecção')
recovery_time_slider = IntSlider(min=1, max=30, step=1, value=default_params['recovery_time'], description='Duração da Doença')
immunity_loss_rate_slider = FloatSlider(min=0.0, max=0.1, step=0.001, value=default_params['immunity_loss_rate'], description='Taxa de Perda de Imunidade')
population_density_slider = FloatSlider(min=0.1, max=1.0, step=0.05, value=default_params['population_density'], description='Densidade Pop.')
movement_rate_slider = FloatSlider(min=0.0, max=1.0, step=0.05, value=default_params['movement_rate'], description='Taxa de Movimento')

# Botão de execução
run_button = Button(description="Iniciar Simulação")

# Crie um widget de saída para os resultados da simulação
simulation_output = widgets.Output()

def on_run_button_clicked(b):
    # O código de simulação agora será executado dentro desta área de saída
    with simulation_output:
        # A primeira coisa é exibir a mensagem de andamento
        display(widgets.Label("Simulação em andamento..."))

        # E então, o código de simulação e visualização é executado
        params = {
            'grid_size': grid_size_slider.value,
            'initial_infected': initial_infected_slider.value,
            'infection_rate': infection_rate_slider.value,
            'recovery_time': recovery_time_slider.value,
            'immunity_loss_rate': immunity_loss_rate_slider.value,
            'num_steps': default_params['num_steps'],
            'population_density': population_density_slider.value,
            'movement_rate': movement_rate_slider.value
        }
        run_simulation_and_display(params)

# Atribui a função ao evento de clique do botão
run_button.on_click(on_run_button_clicked)

# Exibe os widgets em um container, incluindo a área de saída
display(VBox([
    HBox([grid_size_slider, initial_infected_slider]),
    HBox([infection_rate_slider, recovery_time_slider]),
    HBox([immunity_loss_rate_slider, population_density_slider]),
    HBox([movement_rate_slider]),
    run_button,
    simulation_output
]))